# using the Eindhoven representative networks in PowerModelsDistribution.jl 
This is a notebook that demonstrates how to load the representative networks of the Eindhoven dataset using PowerModelsDistribution.jl and perform basic operations.

In [1]:
using Pkg
Pkg.activate(".")

  Activating project at `c:\Users\mnumair\OneDrive - KU Leuven\PhD Agenda\2024\24-10 October\TSK-467 RepoRepFed\Representative-distribution-networks\RDN_Eindhoven_representative_feeders\PMD`


In [2]:
using PowerModelsDistribution
using Ipopt
using FileIO

Here you can choose any of the representative networks of and load it as an ENGINEERING model of PowerModelsDistribution.jl using the `load()` with the file name as argument. below an example for loading the first representative feeder.

In [3]:
RDN1 = load("Eindhoven_RDN1.jld2")

Dict{String, Any} with 11 entries:
  "conductor_ids"  => [1, 2, 3, 4]
  "bus"            => Dict{String, Any}("806"=>Dict{String, Any}("rg"=>[0.0], "…
  "name"           => "rdn1"
  "settings"       => Dict{String, Any}("sbase_default"=>100000.0, "vbases_defa…
  "files"          => ["Eindhoven_RDN1.dss"]
  "voltage_source" => Dict{String, Any}("source"=>Dict{String, Any}("source_id"…
  "line"           => Dict{String, Any}("320"=>Dict{String, Any}("length"=>0.00…
  "data_model"     => ENGINEERING
  "transformer"    => Dict{String, Any}("22"=>Dict{String, Any}("polarity"=>[1,…
  "load"           => Dict{String, Any}("772100000281694"=>Dict{String, Any}("s…
  "linecode"       => Dict{String, Any}("powersystem_db_lv_nayy_4x120_se"=>Dict…

from the ENGINEERING model `RDN1` it is possible to explore the network:
For example, lookup the linecode for the a line in the network:

In [4]:
line_lc = RDN1["line"]["320"]["linecode"]

"powersystem_db_lv_nayy_4x120_se"

Then we can find the details for that linecode from the `linecode` dictionary:

In [5]:
RDN1["linecode"][line_lc] 

Dict{String, Any} with 7 entries:
  "b_fr"  => [132.0 0.0 0.0; 0.0 132.0 0.0; 0.0 0.0 132.0]
  "rs"    => [0.225 0.0 0.0; 0.0 0.225 0.0; 0.0 0.0 0.225]
  "cm_ub" => [600.0, 600.0, 600.0]
  "xs"    => [0.08 0.0 0.0; 0.0 0.08 0.0; 0.0 0.0 0.08]
  "b_to"  => [132.0 0.0 0.0; 0.0 132.0 0.0; 0.0 0.0 132.0]
  "g_to"  => [0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0]
  "g_fr"  => [0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0]

and can see the formatted series resistance matrix for that linecode:

In [6]:
RDN1["linecode"][line_lc]["rs"]

3×3 Matrix{Float64}:
 0.225  0.0    0.0
 0.0    0.225  0.0
 0.0    0.0    0.225

However most of the interesting analysis happens using the MATHEMATICAL model which is used for optimization problems. So we have to transform `RDN1` from the ENGINEERING data model to the MATHEMATICAL model `RDN1_math` 

In [7]:
RDN1_math = transform_data_model(RDN1)

Dict{String, Any} with 18 entries:
  "is_kron_reduced" => true
  "conductor_ids"   => [1, 2, 3]
  "time_elapsed"    => 1.0
  "bus"             => Dict{String, Any}("32"=>Dict{String, Any}("vm_pair_lb"=>…
  "name"            => "rdn1"
  "map"             => Dict{String, Any}[Dict("unmap_function"=>"_map_math2eng_…
  "settings"        => Dict{String, Any}("sbase_default"=>100000.0, "vbases_def…
  "gen"             => Dict{String, Any}("1"=>Dict{String, Any}("pg"=>[0.0, 0.0…
  "branch"          => Dict{String, Any}("32"=>Dict{String, Any}("br_r"=>[11.31…
  "storage"         => Dict{String, Any}()
  "switch"          => Dict{String, Any}()
  "is_projected"    => true
  "per_unit"        => true
  "data_model"      => MATHEMATICAL
  "shunt"           => Dict{String, Any}()
  "transformer"     => Dict{String, Any}("1"=>Dict{String, Any}("source_id"=>"_…
  "bus_lookup"      => Dict{Any, Int64}("806"=>1, "2515"=>2, "2473"=>3, "101"=>…
  "load"            => Dict{String, Any}("24"=>Dict{String,

Then it is possible to perform an optimal power flow (OPF) using `PowerModelsDistribution.jl` functions

In [8]:
RDN1_opf = solve_mc_opf(RDN1_math, ACPUPowerModel, Ipopt.Optimizer)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.17, running with linear solver MUMPS 5.7.3.

Number of nonzeros in equality constraint Jacobian...:     7773
Number of nonzeros in inequality constraint Jacobian.:     1848
Number of nonzeros in Lagrangian Hessian.............:    14964

Total number of variables............................:     1173
                     variables with only lower bounds:      186
                variables with lower and upper bounds:       24
                     variables with only upper bounds:        0
Total number of equality constraints.................:     1173
Total number of inequality c

Dict{String, Any} with 8 entries:
  "solve_time"         => 1.473
  "optimizer"          => "Ipopt"
  "termination_status" => LOCALLY_SOLVED
  "dual_status"        => FEASIBLE_POINT
  "primal_status"      => FEASIBLE_POINT
  "objective"          => 0.431454
  "solution"           => Dict{String, Any}("branch"=>Dict{String, Any}("32"=>D…
  "objective_lb"       => -Inf

Then we can explore the results from the `"solution"` dictionary. FOr example we can find the voltage magnitude `vm` and angle `va` for the three phases of the first bus as follows

In [9]:
RDN1_opf["solution"]["bus"]["1"]

Dict{String, Any} with 2 entries:
  "va" => [-0.0103931, -2.09659, 2.1017]
  "vm" => [0.985095, 1.00724, 0.998472]

### Notes

NOTE 1: All of the above can be done for any of the representative feeders of the dataset, bus simply swapping the file path in the `load()` function.

NOTE 2: This is just an example of how to load the Spanish representative feeders in PowerModelsDistribution.jl. For more information on how to use PowerModelsDistribution.jl package, please refer to the [documentation](https://lanl-ansi.github.io/PowerModelsDistribution.jl/stable/).
